
<br>
============================<br>
Explore 3D images (of cells)<br>
============================<br>
This tutorial is an introduction to three-dimensional image processing. Images<br>
are represented as `numpy` arrays. A single-channel, or grayscale, image is a<br>
2D matrix of pixel intensities of shape ``(n_row, n_col)``, where ``n_row``<br>
(resp. ``n_col``) denotes the number of `rows` (resp. `columns`). We can<br>
construct a 3D volume as a series of 2D `planes`, giving 3D images the shape<br>
``(n_plane, n_row, n_col)``, where ``n_plane`` is the number of planes.<br>
A multichannel, or RGB(A), image has an additional<br>
`channel` dimension in the final position containing color information.<br>
These conventions are summarized in the table below:<br>
=============== =================================<br>
Image type      Coordinates<br>
=============== =================================<br>
2D grayscale    ``[row, column]``<br>
2D multichannel ``[row, column, channel]``<br>
3D grayscale    ``[plane, row, column]``<br>
3D multichannel ``[plane, row, column, channel]``<br>
=============== =================================<br>
Some 3D images are constructed with equal resolution in each dimension (e.g.,<br>
synchrotron tomography or computer-generated rendering of a sphere).<br>
But most experimental data are captured<br>
with a lower resolution in one of the three dimensions, e.g., photographing<br>
thin slices to approximate a 3D structure as a stack of 2D images.<br>
The distance between pixels in each dimension, called spacing, is encoded as a<br>
tuple and is accepted as a parameter by some `skimage` functions and can be<br>
used to adjust contributions to filters.<br>
The data used in this tutorial were provided by the Allen Institute for Cell<br>
Science. They were downsampled by a factor of 4 in the `row` and `column`<br>
dimensions to reduce their size and, hence, computational time. The spacing<br>
information was reported by the microscope used to image the cells.<br>


In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

In [2]:
from skimage import exposure, io, util
from skimage.data import cells3d

###################################################################<br>
Load and display 3D images<br>
==========================

In [3]:
data = util.img_as_float(cells3d()[:, 1, :, :])  # grab just the nuclei

ModuleNotFoundError: The requested file is part of the scikit-image distribution, but requires the installation of an optional dependency, pooch. To install pooch, use your preferred python package manager. Follow installation instruction found at https://scikit-image.org/docs/stable/install.html

In [ ]:
print("shape: {}".format(data.shape))
print("dtype: {}".format(data.dtype))
print("range: ({}, {})".format(data.min(), data.max()))

Report spacing from microscope

In [ ]:
original_spacing = np.array([0.2900000, 0.0650000, 0.0650000])

Account for downsampling of slices by 4

In [ ]:
rescaled_spacing = original_spacing * [1, 4, 4]

Normalize spacing so that pixels are a distance of 1 apart

In [ ]:
spacing = rescaled_spacing / rescaled_spacing[2]

In [ ]:
print("microscope spacing: {}\n".format(original_spacing))
print("rescaled spacing: {} (after downsampling)\n".format(rescaled_spacing))
print("normalized spacing: {}\n".format(spacing))

###################################################################<br>
Let us try and visualize the (3D) image with `io.imshow`.

In [ ]:
try:
    io.imshow(data, cmap="gray")
except TypeError as e:
    print(str(e))

###################################################################<br>
The `io.imshow` function can only display grayscale and RGB(A) 2D images.<br>
We can thus use it to visualize 2D planes. By fixing one axis, we can<br>
observe three different views of the image.

In [ ]:
def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.axis("off")
    if title:
        ax.set_title(title)

In [ ]:
(n_plane, n_row, n_col) = data.shape
_, (a, b, c) = plt.subplots(ncols=3, figsize=(15, 5))

In [ ]:
show_plane(a, data[n_plane // 2], title=f'Plane = {n_plane // 2}')
show_plane(b, data[:, n_row // 2, :], title=f'Row = {n_row // 2}')
show_plane(c, data[:, :, n_col // 2], title=f'Column = {n_col // 2}')

###################################################################<br>
As hinted before, a three-dimensional image can be viewed as a series of<br>
two-dimensional planes. Let us write a helper function, `display`, to<br>
display 30 planes of our data. By default, every other plane is displayed.

In [ ]:
def display(im3d, cmap="gray", step=2):
    _, axes = plt.subplots(nrows=5, ncols=6, figsize=(16, 14))
    vmin = im3d.min()
    vmax = im3d.max()
    for ax, image in zip(axes.flatten(), im3d[::step]):
        ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_xticks([])
        ax.set_yticks([])

In [ ]:
display(data)

###################################################################<br>
Alternatively, we can explore these planes (slices) interactively using<br>
Jupyter widgets. Let the user select which slice to display and show the<br>
position of this slice in the 3D dataset.<br>
Note that you cannot see the Jupyter widget at work in a static HTML page,<br>
as is the case in the scikit-image gallery. For the following piece of<br>
code to work, you need a Jupyter kernel running either locally or in the<br>
cloud: see the bottom of this page to either download the Jupyter notebook<br>
and run it on your computer, or open it directly in Binder.

In [ ]:
def slice_in_3D(ax, i):
    # From https://stackoverflow.com/questions/44881885/python-draw-3d-cube
    Z = np.array([[0, 0, 0],
                  [1, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0],
                  [0, 0, 1],
                  [1, 0, 1],
                  [1, 1, 1],
                  [0, 1, 1]])
    Z = Z * data.shape
    r = [-1, 1]
    X, Y = np.meshgrid(r, r)

    # Plot vertices
    ax.scatter3D(Z[:, 0], Z[:, 1], Z[:, 2])

    # List sides' polygons of figure
    verts = [[Z[0], Z[1], Z[2], Z[3]],
             [Z[4], Z[5], Z[6], Z[7]],
             [Z[0], Z[1], Z[5], Z[4]],
             [Z[2], Z[3], Z[7], Z[6]],
             [Z[1], Z[2], Z[6], Z[5]],
             [Z[4], Z[7], Z[3], Z[0]],
             [Z[2], Z[3], Z[7], Z[6]]]

    # Plot sides
    ax.add_collection3d(
        Poly3DCollection(
            verts,
            facecolors=(0, 1, 1, 0.25),
            linewidths=1,
            edgecolors="darkblue"
        )
    )
    verts = np.array([[[0, 0, 0],
                       [0, 0, 1],
                       [0, 1, 1],
                       [0, 1, 0]]])
    verts = verts * (60, 256, 256)
    verts += [i, 0, 0]
    ax.add_collection3d(
        Poly3DCollection(
            verts,
            facecolors="magenta",
            linewidths=1,
            edgecolors="black"
        )
    )
    ax.set_xlabel("plane")
    ax.set_xlim(0, 100)
    ax.set_ylabel("row")
    ax.set_zlabel("col")

    # Autoscale plot axes
    scaling = np.array([getattr(ax,
                                f'get_{dim}lim')() for dim in "xyz"])
    ax.auto_scale_xyz(* [[np.min(scaling), np.max(scaling)]] * 3)

In [ ]:
def explore_slices(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data)
    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=(20, 5))
        ax_3D = fig.add_subplot(133, projection="3d")
        show_plane(ax, data[plane], title="Plane {}".format(plane), cmap=cmap)
        slice_in_3D(ax_3D, plane)
        plt.show()
    return display_slice

In [ ]:
explore_slices(data);

###################################################################<br>
Adjust exposure<br>
===============<br>
Scikit-image's `exposure` module contains a number of functions for<br>
adjusting image contrast. These functions operate on pixel values.<br>
Generally, image dimensionality or pixel spacing doesn't need to be<br>
considered. In local exposure correction, though, one might want to<br>
adjust the window size to ensure equal size in *real* coordinates along<br>
each axis.

###################################################################<br>
`Gamma correction <https://en.wikipedia.org/wiki/Gamma_correction>`_<br>
brightens or darkens an image. A power-law transform, where `gamma` denotes<br>
the power-law exponent, is applied to each pixel in the image: `gamma < 1`<br>
will brighten an image, while `gamma > 1` will darken an image.

In [ ]:
def plot_hist(ax, data, title=None):
    # Helper function for plotting histograms
    ax.hist(data.ravel(), bins=256)
    ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))
    if title:
        ax.set_title(title)

In [ ]:
gamma_low_val = 0.5
gamma_low = exposure.adjust_gamma(data, gamma=gamma_low_val)

In [ ]:
gamma_high_val = 1.5
gamma_high = exposure.adjust_gamma(data, gamma=gamma_high_val)

In [ ]:
_, ((a, b, c), (d, e, f)) = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

In [ ]:
show_plane(a, data[32], title='Original')
show_plane(b, gamma_low[32], title=f'Gamma = {gamma_low_val}')
show_plane(c, gamma_high[32], title=f'Gamma = {gamma_high_val}')

In [ ]:
plot_hist(d, data)
plot_hist(e, gamma_low)
plot_hist(f, gamma_high)
# sphinx_gallery_thumbnail_number = 4

###################################################################<br>
`Histogram<br>
equalization <https://en.wikipedia.org/wiki/Histogram_equalization>`_<br>
improves contrast in an image by redistributing pixel intensities. The most<br>
common pixel intensities get spread out, increasing contrast in low-contrast<br>
areas. One downside of this approach is that it may enhance background<br>
noise.

In [ ]:
equalized_data = exposure.equalize_hist(data)

In [ ]:
display(equalized_data)

###################################################################<br>
As before, if we have a Jupyter kernel running, we can explore the above<br>
slices interactively.

In [ ]:
explore_slices(equalized_data);

###################################################################<br>
Let us now plot the image histogram before and after histogram equalization.<br>
Below, we plot the respective cumulative distribution functions (CDF).

In [ ]:
_, ((a, b), (c, d)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 8))

In [ ]:
plot_hist(a, data, title="Original histogram")
plot_hist(b, equalized_data, title="Equalized histogram")

In [ ]:
cdf, bins = exposure.cumulative_distribution(data.ravel())
c.plot(bins, cdf, "r")
c.set_title("Original CDF")

In [ ]:
cdf, bins = exposure.cumulative_distribution(equalized_data.ravel())
d.plot(bins, cdf, "r")
d.set_title("Histogram equalization CDF")

###################################################################<br>
Most experimental images are affected by salt and pepper noise. A few bright<br>
artifacts can decrease the relative intensity of the pixels of interest. A<br>
simple way to improve contrast is to clip the pixel values on the lowest and<br>
highest extremes. Clipping the darkest and brightest 0.5% of pixels will<br>
increase the overall contrast of the image.

In [ ]:
vmin, vmax = np.percentile(data, q=(0.5, 99.5))

In [ ]:
clipped_data = exposure.rescale_intensity(
    data,
    in_range=(vmin, vmax),
    out_range=np.float32
)

In [ ]:
display(clipped_data)